In [15]:
import pandas as pd
import numpy as np
from scipy.stats import shapiro, friedmanchisquare, wilcoxon, levene
from statsmodels.stats.anova import AnovaRM
from statsmodels.stats.multitest import multipletests

In [26]:
accuracy_df  = pd.read_csv("results\\acc_results.csv")
precision_df = pd.read_csv("results\\prec_results.csv")
recall_df    = pd.read_csv("results\\recall_results.csv")
f1_df        = pd.read_csv("results\\f1_results.csv")
acc_cnn      = pd.read_csv("results\\acc_results_cnn.csv")

metrics = {"Accuracy": accuracy_df,
           "Precision": precision_df,
           "Recall": recall_df,
           "F1": f1_df}

In [18]:
def test_metrics(df, metric_name):
    print(f"\n===== {metric_name} =====")

    cnn = df['wlasny'].values
    resnet = df['resnet'].values
    efficient = df['efficient'].values

    # ===== sprawdzenie normalności rozkładu =====
    print("Shapiro-Wilk test (normalność rozkładu)")
    normal_flags = []
    for name, data in zip(['CNN','ResNet','EfficientNet'], [cnn,resnet,efficient]):
        stat, p = shapiro(data)
        normal = p > 0.05
        normal_flags.append(normal)
        print(f"{name}: stat={stat:.4f}, p={p:.4f} -> {'normalny' if normal else 'nienormalny'}")

    # ===== sprawdzenie homogeniczności wariancji =====
    stat, p = levene(cnn, resnet, efficient)
    homo = p > 0.05
    print(f"\nTest Levene’a: stat={stat:.4f}, p={p:.4f} -> {'jednorodne' if homo else 'niejednorodne'}")

    # ===== wybór testu =====
    if all(normal_flags) and homo:
        print("\n>>> Wszystkie warunki spełnione -> ANOVA (parametryczny)")
        # ANOVA powtarzanych pomiarów wymaga formatu "long"
        df = pd.DataFrame({
            "id": np.tile(np.arange(len(cnn)), 3),
            "model": np.repeat(["CNN","ResNet","EfficientNet"], len(cnn)),
            "score": np.concatenate([cnn, resnet, efficient])
        })
        anova = AnovaRM(df, 'score', 'id', within=['model']).fit()
        print(anova)
    else:
        print("\n>>> Warunki niespełnione -> używamy testu Friedmana (nieparametryczny)")
        stat, p = friedmanchisquare(cnn, resnet, efficient)
        print(f"Friedman: stat={stat:.4f}, p={p:.4f}")

        print("\nPost-hoc Wilcoxon (z poprawką Holm-Bonferroni):")
        pairs = [("CNN", "ResNet", cnn, resnet),
                 ("CNN", "EfficientNet", cnn, efficient),
                 ("ResNet", "EfficientNet", resnet, efficient)]

        # obliczamy p-value dla wszystkich par
        labels = []
        p_values = []
        stats_list = []
        for name1, name2, d1, d2 in pairs:
            stat_w, p_val = wilcoxon(d1, d2)
            labels.append(f"{name1} vs {name2}")
            p_values.append(p_val)
            stats_list.append(stat_w)

        # korekta Holm-Bonferroni
        rejected, p_corrected, _, _ = multipletests(p_values, method="holm")

        # raport wyników
        for lbl, stat_w, p_raw, p_corr, rej in zip(labels, stats_list, p_values, p_corrected, rejected):
            signif = "istotna" if rej else "brak istotności"
            print(f"{lbl}: stat={stat_w:.4f}, raw p={p_raw:.4f}, corrected p={p_corr:.4f} -> {signif}")

In [27]:
for metric_name, df in metrics.items():
    test_metrics(df, metric_name)


===== Accuracy =====
Shapiro-Wilk test (normalność rozkładu)
CNN: stat=0.8559, p=0.2139 -> normalny
ResNet: stat=0.9316, p=0.6071 -> normalny
EfficientNet: stat=0.9556, p=0.7773 -> normalny

Test Levene’a: stat=3.3308, p=0.0707 -> jednorodne

>>> Wszystkie warunki spełnione -> ANOVA (parametryczny)
              Anova
      F Value Num DF Den DF Pr > F
----------------------------------
model  0.8298 2.0000 8.0000 0.4705


===== Precision =====
Shapiro-Wilk test (normalność rozkładu)
CNN: stat=0.9000, p=0.4099 -> normalny
ResNet: stat=0.5522, p=0.0001 -> nienormalny
EfficientNet: stat=0.8423, p=0.1714 -> normalny

Test Levene’a: stat=1.3800, p=0.2888 -> jednorodne

>>> Warunki niespełnione -> używamy testu Friedmana (nieparametryczny)
Friedman: stat=8.3158, p=0.0156

Post-hoc Wilcoxon (z poprawką Holm-Bonferroni):
CNN vs ResNet: stat=0.0000, raw p=0.0625, corrected p=0.1875 -> brak istotności
CNN vs EfficientNet: stat=0.0000, raw p=0.0625, corrected p=0.1875 -> brak istotności
ResNet 